In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [ ]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

In [ ]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [191]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [194]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artist
from plan import validate_plan

tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, search_tool, find_similar_artist]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Manual Test of Tool Infra

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_playlists",
            "args" : {},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

# tool_node.invoke({"messages": [message_with_single_tool_call]})

# Bind Tools to Model

In [195]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=0.8)
llm_with_tools = llm.bind_tools(tools)

# Define Prompt

In [196]:
prompt = """
Create and execute a step-by-step plan with atomic tasks to solve the following problem:

**Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.

**Rules:**

1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**

2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**

3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.

4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.

5. **Recommend 3-4 tracks for each new artist.

6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, considering tempo, energy, and mood, using best practices.

7. **Playlist Length:** Ensure the new playlist contains at least 100 tracks.

**Instructions:**

- Follow the rules strictly to meet the objective.
- Prefer using a tool if available, instead of your Knowledge Base
- In your responses always state the step you are working on
- Step(s) should be repeated any number of times to achieve the desired outcome.
- Do not ask for user confirmation
"""


# First Message

In [197]:
from langchain_core.messages import HumanMessage

human_message = HumanMessage(prompt)
ai_tool_call_message = llm_with_tools.invoke([human_message])

2024-11-12 19:06:09,726 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [198]:
ai_tool_call_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  validate_plan (call_TwCeO2eE8kt9x9gbxvGFfaur)
 Call ID: call_TwCeO2eE8kt9x9gbxvGFfaur
  Args:
    plan: {'steps': [{'name': 'get_artists', 'description': "Retrieve the list of unique artists from the 'New Rock and Blues' playlist.", 'success_criteria': "Successfully obtained a list of all unique artists from the 'New Rock and Blues' playlist.", 'tool': 'functions.get_artists_from_playlist', 'action': "Call the 'get_artists_from_playlist' function with the playlist ID of 'New Rock and Blues'."}, {'name': 'find_similar_artists', 'description': "Find 3-4 similar artists for each artist in the 'New Rock and Blues' playlist.", 'success_criteria': 'A list of similar artists for each original artist is obtained, with each artist having 3-4 similar artists listed.', 'tool': 'functions.find_similar_artist', 'action': "Call the 'find_similar_artist' function for each artist obtained in the previous ste

In [199]:
import json
plan = ai_tool_call_message.additional_kwargs["tool_calls"][0]["function"]["arguments"]
json.loads(plan)

{'plan': {'steps': [{'name': 'get_artists',
    'description': "Retrieve the list of unique artists from the 'New Rock and Blues' playlist.",
    'success_criteria': "Successfully obtained a list of all unique artists from the 'New Rock and Blues' playlist.",
    'tool': 'functions.get_artists_from_playlist',
    'action': "Call the 'get_artists_from_playlist' function with the playlist ID of 'New Rock and Blues'."},
   {'name': 'find_similar_artists',
    'description': "Find 3-4 similar artists for each artist in the 'New Rock and Blues' playlist.",
    'success_criteria': 'A list of similar artists for each original artist is obtained, with each artist having 3-4 similar artists listed.',
    'tool': 'functions.find_similar_artist',
    'action': "Call the 'find_similar_artist' function for each artist obtained in the previous step."},
   {'name': 'filter_existing_artists',
    'description': "Remove artists already present in the 'New Rock and Blues' Playlist from the list of new s

# chat_prompt_template holds all messages (Human, AI, Tool)

In [200]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template: ChatPromptTemplate = human_message + ai_tool_call_message
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n**Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.\n\n**Rules:**\n\n1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.\n\n5. **Recommend 3-4 tracks for each new artist.\n\n6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, considering tempo, energy, and mood, using best practices.\n\n7. **Playlist Length:*

# Check for Tools in Message

In [201]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0 or len(tools_in_ai_message) > 1:
    raise ValueError
else:
    print(tools_in_ai_message)

[StructuredTool(name='validate_plan', description='Validates a step-by-step plan to solve a problem\n\nArgs:\n    plan (Plan): a step-by-step plan to solve a problem\n\nReturns:\n    bool: whether plan is okay or not', args_schema=<class 'langchain_core.utils.pydantic.validate_plan'>, func=<function validate_plan at 0x0000024E12C600E0>)]


# Tool Call (should be validate_plan())

In [202]:
response = tool_node.invoke({"messages": messages})
tool_message = response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: validate_plan

true


In [203]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n**Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.\n\n**Rules:**\n\n1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.\n\n5. **Recommend 3-4 tracks for each new artist.\n\n6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, considering tempo, energy, and mood, using best practices.\n\n7. **Playlist Length:*

# Send tool result to LLM

In [204]:
ai_tool_call_message = llm_with_tools.invoke(chat_prompt_template.format_messages())
ai_tool_call_message.pretty_print()

2024-11-12 19:06:48,403 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

The plan has been validated and is ready for execution. Let's begin with Step 1.

### Step 1: Get Artists
**Objective:** Retrieve the list of unique artists from the 'New Rock and Blues' playlist.

I will start by calling the function to get the unique artists from the 'New Rock and Blues' playlist.
Tool Calls:
  get_playlists (call_53p08hL3fpPRh29PfS3QZpMd)
 Call ID: call_53p08hL3fpPRh29PfS3QZpMd
  Args:


# Tool Call (should be get_playlists())

In [205]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()
response = tool_node.invoke({"messages": messages})

In [206]:
playlist_tool_message: ToolMessage = response["messages"][0]
playlist_tool_message.pretty_print()

================================= Tool Message =================================
Name: get_playlists

{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "descripti

In [ ]:
# We need to check that is JSON serializable
# json.loads(playlist_tool_message.content)

# Format Message List for LLM

In [207]:
chat_prompt_template += playlist_tool_message
messages = chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [208]:
ai_tool_call_message = llm_with_tools.invoke(messages)
ai_tool_call_message.pretty_print()

2024-11-12 19:07:26,587 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_artists_from_playlist (call_N9e37UeO7HKX2J91mX9HcBRe)
 Call ID: call_N9e37UeO7HKX2J91mX9HcBRe
  Args:
    playlist_id: spotify:playlist:01tk0aitEuGK0ajWCkzdKc


In [209]:
chat_prompt_template += ai_tool_call_message
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [210]:
response = tool_node.invoke({"messages": messages})
track_list_tool_message: ToolMessage = response["messages"][0]

# Decode AI Tool Call message

In [211]:
track_list_tool_message.pretty_print()

================================= Tool Message =================================
Name: get_artists_from_playlist

{'artists': {'Jackyl', 'The Blue Stones', 'Blur', 'Post Malone', 'Wolfmother', 'The Weeknd', 'Audioslave', 'Lady Gaga', 'Chris Cornell', 'Florence + The Machine', 'Maggie Bjorklund', 'Welshly Arms', 'Joywave', 'The Arcs', 'Robert Randolph & The Family Band', 'The Bones of J.R. Jones', 'Rival Sons', 'Jonah Tolchin', 'Reignwolf', 'Talking Heads', 'Chris Stapleton', 'Foo Fighters', 'Danielle Ponder', 'Hozier', 'Cage The Elephant', 'Santana', 'Gary Clark Jr.', 'Everlast', 'Broken Witt Rebels', 'The Rolling Stones', 'Yeah Yeah Yeahs', 'Colter Wall', 'Kings of Leon', 'Uncle Lucius', 'The Struts', 'Thirty Seconds To Mars', 'Stone Sour', 'Amy Winehouse', 'Whiskey Myers', 'James', 'Shakey Graves', 'Last Train', 'Arcade Fire', 'Propellerheads', 'The Black Keys', 'Blackberry Smoke', 'Goodbye June', 'The Heavy', 'Morningsiders', 'Mark Ronson', 'Sons Of The East', 'Inhaler', 'Dan Auerba

In [ ]:
# json.loads(track_list_tool_message.content)

# Add Tool message containing playlist tracks to list of messages

In [212]:
chat_prompt_template += track_list_tool_message
messages = chat_prompt_template.format_messages()
messages

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\n**Objective:** Build a Spotify playlist with tracks that have the same vibe and genres as my 'New Rock and Blues' playlist, following the rules below.\n\n**Rules:**\n\n1. **For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of the similar music style**\n\n2 .**Remove from the new artist list those that are already present in the 'New Rock and Blues' Playlist**\n\n3. **Focus on Post-2010 Success:** Only include tracks from artists who achieved success after the year 2010.\n\n4. **Minimum Number of New Artists:** Include tracks from at least 40 different artists not present in the 'New Rock and Blues' playlist.\n\n5. **Recommend 3-4 tracks for each new artist.\n\n6. **Arrange for Smooth Listening Experience:** Organize the tracks to create a smooth listening experience, considering tempo, energy, and mood, using best practices.\n\n7. **Playlist Length:*

# Send Tool Result to LLM

In [213]:
# llm_with_structured_output = llm.with_structured_output(schema=Tracks, method='function_calling', include_raw=True)
# response = llm_with_structured_output.invoke(chat_prompt_template.format_messages())
# response
response = llm_with_tools.invoke(messages)

2024-11-12 19:28:50,931 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [215]:
response.pretty_print()

================================== Ai Message ==================================

### Step 1 Completed: Get Artists
I have successfully retrieved the list of unique artists from the 'New Rock and Blues' playlist.

### Step 2: Find Similar Artists
**Objective:** For each artist in the 'New Rock and Blues' playlist, find 3-4 artists of a similar music style.

I will now proceed to find similar artists for each artist from the list. This will take a moment as I search for each one. Let's go ahead and find similar artists.
Tool Calls:
  find_similar_artist (call_YIVxGfzs2Ut8WPzlpfxP9hpe)
 Call ID: call_YIVxGfzs2Ut8WPzlpfxP9hpe
  Args:
    artist: spotify:artist:Jackyl
  find_similar_artist (call_kCKLDyB7eGrofohT65vfPMlb)
 Call ID: call_kCKLDyB7eGrofohT65vfPMlb
  Args:
    artist: spotify:artist:The Blue Stones
  find_similar_artist (call_yvFK18z8C7cAtTcZ4EZebb5F)
 Call ID: call_yvFK18z8C7cAtTcZ4EZebb5F
  Args:
    artist: spotify:artist:Blur
  find_similar_artist (call_aEa3ERepnQuRl21drluc

In [ ]:
chat_prompt_template += response
messages = chat_prompt_template.format_messages()

In [ ]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
print(tools_in_ai_message)
if len(tools_in_ai_message) == 0 or len(tools_in_ai_message) > 1:
    raise ValueError

# Tool Call (check_artists)

In [ ]:
response = tool_node.invoke({"messages": messages})

In [ ]:
check_artist_tool_message: ToolMessage = response["messages"][0]
check_artist_tool_message.pretty_print()

In [ ]:
# We recreate the ToolMessage to take care of UNicode characters
tool_message = ToolMessage(content=check_artist_tool_message.content, name=check_artist_tool_message.name, tool_call_id=check_artist_tool_message.tool_call_id)
tool_message.pretty_print()

In [ ]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

In [ ]:
response = llm_with_tools.invoke(chat_prompt_template.format_messages())

In [ ]:
response.pretty_print()

In [ ]:
chat_prompt_template += response
chat_prompt_template.format_messages()

In [ ]:
response = tool_node.invoke({"messages": [response]})

In [ ]:
response